<a href="https://colab.research.google.com/github/nsydn/isu_kis_okulu/blob/main/supply_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Supply Network**

<img src="https://github.com/nsydn/kis_okulu/blob/main/image3.png?raw=1" alt="Drawing" style="width: 600px;"/>

### **Problem tanımı** (Problem Definition)

Uluslararası bir üretim şirketinin Lojistik Ağı Yöneticisi olarak, taşıma maliyetlerinde son zamanlarda görülen artışı ve gelecekteki talep tahminlerini göz önünde bulundurarak şirketin lojistik ağını önümüzdeki 5 yıl için yeniden tasarlamak istiyorsunuz.

#### **Ağ yapısı** (Network Structure)
* Brezilya, ABD, Hindistan, Japonya, Almanya olmak üzere 5 ülkede üretim ve satış yapıyoruz
* Her ülkede 2 tip üretim tesisi kurulabilir: düşük ve yüksek kapasite
* Taşıma maliyetleri (USD/konteyner)
* Piyasa talebi (Adet/yıl)

#### **Maliyetler** (Costs)
* Sabit maliyetler (makina/ekipman, kira, vb.): $f$
* Üretim değişken maliyetleri (üretim hattı operatörleri, hammadde, vb.): $v_1$
* Taşıma değişken maliyetleri (1 konteynır = 1000 adet): $v_2$

### **Bilgisayar Modeli** (Computer Model)

In [ ]:
%pip install pulp
import pandas as pd
from pulp import *

In [33]:
# upload data
from google.colab import files
uploaded = files.upload()

Saving data.xlsx to data (1).xlsx


In [82]:
# read data
fixed_costs = pd.read_excel('data.xlsx', 'fixed_costs',index_col = 0)
manuf_costs = pd.read_excel('data.xlsx', 'manuf_costs', index_col = 0) #per_item
freight_costs = pd.read_excel('data.xlsx', 'freight_costs', index_col = 0)/1000 #per_container
var_costs = manuf_costs + freight_costs
capacities = pd.read_excel('data.xlsx', 'capacities', index_col = 0)
demands = pd.read_excel('data.xlsx', 'demands', index_col = 0) #per_month

In [36]:
# introduce locations and facility types
locations = ['US', 'Germany', 'Japan', 'Brazil', 'India']
ftypes = ['low', 'high']

In [59]:
# initiate the model
my_model = LpProblem('network',LpMinimize)

#### **Karar değişkenleri** (Decision variables)
* $x_{i,j}$: Lokasyon $i$'den lokasyon $j$'ye transfer miktarı ($i,j\in$ `locations`)
* $y_{i,k}$: Lokasyon $i$'de $k$ kapasiteli tesis kurulacaksa 1, yoksa 0 ($i\in$ `locations`, $k\in$ `ftypes`)

In [60]:
x = LpVariable.dicts('x', [(i,j) for i in locations for j in locations], lowBound=0, upBound=None, cat='Continuous')
y = LpVariable.dicts('y', [(i,k) for i in locations for k in ftypes], cat='Binary')

#### **Amaç fonksiyonu** (Objective function)
Toplam tesis kurma ve dağıtım maliyetlerini minimize edecek bir dağıtım ağı kurmak istiyoruz:

$$\sum_{i} \sum_{k} f_{i,t} y_{i,k} + \sum_{i} \sum_{j} v_{i,j} x_{i,j}$$

In [61]:
my_model += (lpSum([fixed_costs.loc[i,k] * y[i,k] for i in locations for k in ftypes]) 
          + lpSum([var_costs.loc[i,j] * x[i,j] for i in locations for j in locations]))

#### **Kısıtlar** (Constraints)
* Her bir $i$ lokasyonunda yalnızca bir tip tesis kurabiliriz (`low` ya da `high`):
$$\sum_{k} y_{i,k} \leq 1\quad ∀i$$
* Herbir $i$ lokasyonundan çıkan toplam transferin o lokasyondaki kapasiteyi aşmadığından emin olmalıyız: $$\sum_{j} x_{i,j} \leq \sum_{k} c_i y_k \quad ∀i$$
* Herbir $j$ lokasyonuna gelen toplam transferin o lokasyondaki talebi tam olarak karşıladığından emin olmalıyız: $$\sum_{i} x_{i,j} = d_j \quad ∀j$$

In [62]:
for i in locations:
  my_model += lpSum([y[i,k] for k in ftypes]) <= 1
for i in locations:
  my_model += lpSum([x[i,j] for j in locations]) <= lpSum([capacities.loc[i,k]*y[i,k] for k in ftypes])
for j in locations:
  my_model += lpSum([x[i,j] for i in locations]) == demands.loc[j,'demand']

#### **Modelin çözümü** (Solving the model)

In [63]:
my_model.solve()
print('Total cost = {:,} ($/month)'.format(int(value(my_model.objective))))
print("Solution status: {}".format(LpStatus[my_model.status]))

Total cost = 67,768,475 ($/month)
Solution status: Optimal


#### **Sonuçlar** (Results)
* Hangi lokasyona hangi kapasitede bir tesis açıyoruz?
* Hangi lokasyonlar arası ne kadar ürün transfer ediyoruz?

In [64]:
plants = fixed_costs
for i in locations:
  for k in ftypes:
    plants.loc[i,k] = y[i,k].varValue
plants

,low,high
location,,
US,0,0
Germany,1,0
Japan,0,1
Brazil,0,1
India,0,1


In [79]:
transfers = freight_costs
for i in locations:
  for j in locations:
    transfers.loc[i,j] = x[i,j].varValue
transfers  

,US,Germany,Japan,Brazil,India
location,,,,,
US,0.0,0.0,0.0,0.0,0.0
Germany,105000.0,90000.0,200000.0,0.0,0.0
Japan,0.0,0.0,1500000.0,0.0,0.0
Brazil,1355000.0,0.0,0.0,145000.0,0.0
India,1340000.0,0.0,0.0,0.0,160000.0


#### Senaryo A: Üretimi işgücü maliyeti düşük olan bölgeye kaydırma
Hindistan'daki yüksek kapasiteli fabrikanın kapasitesini 2 katına çıkarırsak ne olur? (Sabit maliyetin de 2 katına çıktığını varsayalım.)

In [85]:
capacities.loc['India','high'] *= 2
my_model = LpProblem('network',LpMinimize)
x = LpVariable.dicts('x', [(i,j) for i in locations for j in locations], lowBound=0, upBound=None, cat='Continuous')
y = LpVariable.dicts('y', [(i,k) for i in locations for k in ftypes], cat='Binary')
my_model += (lpSum([fixed_costs.loc[i,k] * y[i,k] for i in locations for k in ftypes]) 
          + lpSum([var_costs.loc[i,j] * x[i,j] for i in locations for j in locations]))
for i in locations:
  my_model += lpSum([y[i,k] for k in ftypes]) <= 1
for i in locations:
  my_model += lpSum([x[i,j] for j in locations]) <= lpSum([capacities.loc[i,k]*y[i,k] for k in ftypes])
for j in locations:
  my_model += lpSum([x[i,j] for i in locations]) == demands.loc[j,'demand']
my_model.solve()
capacities.loc['India','high'] *= 0.5
print('Total cost = {:,} ($/month)'.format(int(value(my_model.objective))))
print("Solution status: {}".format(LpStatus[my_model.status]))

Total cost = 55,028,000 ($/month)
Solution status: Optimal


In [86]:
transfers = freight_costs
for i in locations:
  for j in locations:
    transfers.loc[i,j] = x[i,j].varValue
transfers  

,US,Germany,Japan,Brazil,India
location,,,,,
US,0.0,0.0,0.0,0.0,0.0
Germany,0.0,0.0,0.0,0.0,0.0
Japan,0.0,0.0,1500000.0,0.0,0.0
Brazil,250000.0,0.0,0.0,145000.0,0.0
India,2550000.0,90000.0,200000.0,0.0,160000.0


#### Senaryo B: Konteynır kısıtı nedeniyle artan taşıma maliyetleri
Konteynır kısıtı nedeniyle taşıma maliyetleri 5 katına çıkarsa ne olur?

In [87]:
capacities.loc['India','high'] *= 0.5
freight_costs *= 5
my_model = LpProblem('network',LpMinimize)
x = LpVariable.dicts('x', [(i,j) for i in locations for j in locations], lowBound=0, upBound=None, cat='Continuous')
y = LpVariable.dicts('y', [(i,k) for i in locations for k in ftypes], cat='Binary')
my_model += (lpSum([fixed_costs.loc[i,k] * y[i,k] for i in locations for k in ftypes]) 
          + lpSum([var_costs.loc[i,j] * x[i,j] for i in locations for j in locations]))
for i in locations:
  my_model += lpSum([y[i,k] for k in ftypes]) <= 1
for i in locations:
  my_model += lpSum([x[i,j] for j in locations]) <= lpSum([capacities.loc[i,k]*y[i,k] for k in ftypes])
for j in locations:
  my_model += lpSum([x[i,j] for i in locations]) == demands.loc[j,'demand']
my_model.solve()
print('Total cost = {:,} ($/month)'.format(int(value(my_model.objective))))
print("Solution status: {}".format(LpStatus[my_model.status]))

Total cost = 67,768,475 ($/month)
Solution status: Optimal


In [88]:
transfers = freight_costs
for i in locations:
  for j in locations:
    transfers.loc[i,j] = x[i,j].varValue
transfers  

,US,Germany,Japan,Brazil,India
location,,,,,
US,0.0,0.0,0.0,0.0,0.0
Germany,105000.0,90000.0,200000.0,0.0,0.0
Japan,0.0,0.0,1500000.0,0.0,0.0
Brazil,1355000.0,0.0,0.0,145000.0,0.0
India,1340000.0,0.0,0.0,0.0,160000.0


In [90]:
pd.DataFrame(index = locations, columns = locations)

,US,Germany,Japan,Brazil,India
US,NaN,NaN,NaN,NaN,NaN
Germany,NaN,NaN,NaN,NaN,NaN
Japan,NaN,NaN,NaN,NaN,NaN
Brazil,NaN,NaN,NaN,NaN,NaN
India,NaN,NaN,NaN,NaN,NaN
